In [ ]:
"""
author: Bowen Chen
mail: bowenc@sfu.ca
Contact me if there is any issues.

This is the notebook as a guide on how to use the script to regenerate the data visualization part in the porject.
Include:
1. number of patient vs the day they live for each cytokine
2. mahanttan plot of the GWAS result
"""

In [ ]:
# 1. number of patient vs the day they live for each cytokine. See cytokine_visual.py

In [ ]:
import os
import sys
import math
script_dir = os.path.split(os.path.realpath(__file__))[0]
root_path = os.path.dirname(script_dir)
data_path = os.path.join(root_path, "data/")
export_path = os.path.join(root_path, "visualization/cytokineVSpatient")
if not os.path.exists(export_path):
    os.mkdir(export_path)
# those path can be manually changed

In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel(os.path.join(data_path, 'Sepsis_patient_cytokine_levels.xlsx'), sheet_name='Sheet2')
# read in data from excel

In [ ]:
import numpy as np
import matplotlib 
from sys import platform as _platform
if _platform == "darwin":
    # MAC OS X
    matplotlib.use('TkAgg') 
import matplotlib.pyplot as pyplt
import matplotlib.colors as colors
import matplotlib.axis as axis
import matplotlib.ticker as ticker

pyplt.style.use('ggplot')

# plot number of patient in each cytokine level
for cytokine in df.columns[1:]:
    data_list = [] # {cytokine_level:numer of patient}
    for index in df.index:
        data = df[cytokine][index]
        if math.isnan(data):
            continue
        data_list.append(data)
    fig = pyplt.figure()
    ax = fig.add_subplot(111)
    n, bins, patches = ax.hist(data_list, bins=len(data_list))
    # color the plot
    fracs = n / n.max()
    norm = colors.Normalize(fracs.min(), fracs.max())
    for thisfrac, thispatch in zip(fracs, patches):
        color = pyplt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    pyplt.xlabel('cytokine level')
    pyplt.ylabel('#patient')
    pyplt.title('Histogram - ' + cytokine)
    pyplt.grid(True)
    fig.savefig(os.path.join(export_path, cytokine + '.png'))

In [ ]:
%%bash
# if you want to run the script above in the bash shell
# use python3
# dependency like pandas and matplotlib needs to be preinstalled
python cytokine_visual.py

In [ ]:
# 2. generate mahanttan plot of the GWAS result

In [ ]:
%%R
input_file = "data/EGF.assoc"
output_file = "data/manhattan_GWAS/EGF_GWAS.png"
# to detect whether the pkg is preinstalled
is.installed <- function(mypkg) is.element(mypkg, installed.packages()[,1]) 
if (!is.installed('qqman')){
    install.packages('qqman', repo='http://cran.r-project.org') # This only needs to be done once
    # try install.packages('qqman') without repo if not work
}
data<-read.table(file=input_file, header=T)
library('qqman')
data<-na.omit(data) # drop NaN's
png(filename=output_file)
manhattan(data)
dev.off()

In [ ]:
%%bash
# if you want to run the script in the bash shell
# see script version manhattan.R
# example of use
Rscript --vanilla manhattan.R data/EGF.assoc data/manhattan_GWAS/EGF_GWAS.png